In [9]:
from pylab import *
import numpy as np
import numba

In [18]:
def unpack_mode2(dwords, d):
    for samp in xrange(4):
        # convert from 4-bit two's complement to int8
        d[samp::4, :, 0] = (dwords & 0xf) - (dwords & 0x8)*2 # real
        dwords >>= 4
        d[samp::4, :, 1] = (dwords & 0xf) - (dwords & 0x8)*2 # imag
        dwords >>= 4
        
    return d
    
    
@numba.jit(nopython=True)
def unpack_mode2_jit(dwords, d):
    for samp in xrange(4):
        # convert from 4-bit two's complement to int8
        d[samp::4, :, 0] = (dwords & 0xf) - (dwords & 0x8)*2 # real
        dwords >>= 4 # numba doesn't support this - so it fails.
        d[samp::4, :, 1] = (dwords & 0xf) - (dwords & 0x8)*2 # imag
        dwords >>= 4
        
    return d 

@numba.jit(nopython=True)
def unpack_mode2_jit_v2(dwords, d):
    (nsampwords, nchan) = dwords.shape
    for s in xrange(nsampwords):
        for c in xrange(nchan):
            word = dwords[s, c]
            for samp in xrange(4):
                d[4*s + samp, c, 0] = (word & 0xf) - (word & 0x8)*2 # real
                word >>= 4
                d[4*s + samp, c, 1] = (word & 0xf) - (word & 0x8)*2 # imag
                word >>= 4
    return d


        
        


In [19]:
mode = 2
nchan = 8 # number of channels
nsamps = 4096 # number of time samples
nbits = 8 # bits per sample
nwords = nsamps*nchan * nbits / 32 # number of 32 bit words
dwords = np.arange(nwords, dtype=np.uint32)
dwords=np.random.randint(1<<31, size=nwords, dtype=np.uint32)
dwords.shape = (nsamps*nbits/32, nchan)
d = np.empty((nsamps, nchan, 2), dtype=np.int8)


In [20]:
din = dwords.copy()
%timeit unpack_mode2(din, d)

1000 loops, best of 3: 304 µs per loop


In [21]:
din = dwords.copy()
%timeit  unpck_mode2_jit_v2(din, d)

10000 loops, best of 3: 40.5 µs per loop


In [22]:
res1 = unpack_mode2(dwords.copy(), d)
res2 = unpack_mode2_jit_v2(dwords.copy(), d)
np.all(res1 == res2)


True

In [23]:
print res2

[[[-5 -4]
  [ 7 -4]
  [ 5 -3]
  ...
  [-6 -6]
  [ 6  6]
  [-2 -8]]

 [[-6 -3]
  [-8  7]
  [ 3  5]
  ...
  [ 6  3]
  [ 4 -2]
  [ 4 -7]]

 [[ 0  7]
  [ 2  4]
  [-7 -5]
  ...
  [ 1  2]
  [ 7  1]
  [-3 -5]]

 ...

 [[-1  7]
  [ 5 -8]
  [ 1 -1]
  ...
  [-5 -8]
  [-1 -4]
  [-2  0]]

 [[ 5  0]
  [-1  4]
  [-8  0]
  ...
  [ 1  6]
  [ 5 -2]
  [-8 -7]]

 [[ 4  6]
  [ 3  0]
  [ 0  7]
  ...
  [-6  5]
  [-4  1]
  [ 1  5]]]


In [24]:
np.random.randint?